In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [4]:
'''
Cleaning Data:

-making Transforms for preprocessing
-specifing the dataset and their diretory, then applying the transforms on them.
-passing transforms to data loaders that pack the datasets into batches
'''
data_dir = 'path_for_dataset'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
valid_data = datasets.ImageFolder(data_dir + '/valid', transform=test_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=100, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100)

In [5]:
'''
Using the pretrained model on Imagenet Dataset(a huge dataset with 1000 different classes)
then turning off their parameters for training(they won't be pdated if trained)
then removing the classifier layers(the fully connected layers)
finally, adding our new classifier to the network which has 3 outputs only not 1000
'''
model= models.resnet152(pretrained = True)

for param in model.parameters():
    param.requires_grad = False
    

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 512)),
                          ('relu', nn.ReLU()),
                          ('Dropout',nn.Dropout(0.45)),
                          ('fc2', nn.Linear(512, 3)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

model.fc = classifier

In [6]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    model.cuda()

In [7]:
'''
using Adam as an optimiser as it has a momentum, we don't want to be stuck in a local minima.
adn using Negative Likelihood Log Loss function for the criterion. note that the finall layer is log softmax

'''
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.NLLLoss()

# specify optimizer
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [16]:
model.load_state_dict(torch.load('temp_model.pt'))

In [17]:
## number of epochs to train the model
n_epochs = 30

#valid_loss_min = np.Inf # track change in validation loss
#valid_loss_min =0.6697391470273336
for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(valid_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    

    torch.save(model.state_dict(), "temp_model.pt")  
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
       # model_name=  "model_with_loss=_"+str(valid_loss)+".pt"
        torch.save(model.state_dict(), 'model_dermatology.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.587527 	Validation Loss: 0.684956
Epoch: 2 	Training Loss: 0.580916 	Validation Loss: 0.695845
Epoch: 3 	Training Loss: 0.570567 	Validation Loss: 0.691252
Epoch: 4 	Training Loss: 0.584392 	Validation Loss: 0.748235
Epoch: 5 	Training Loss: 0.587244 	Validation Loss: 0.702227
Epoch: 6 	Training Loss: 0.579297 	Validation Loss: 0.684876
Epoch: 7 	Training Loss: 0.570376 	Validation Loss: 0.715707
Epoch: 8 	Training Loss: 0.585352 	Validation Loss: 0.711866
Epoch: 9 	Training Loss: 0.563048 	Validation Loss: 0.661966
Validation loss decreased (0.669739 --> 0.661966).  Saving model ...
Epoch: 10 	Training Loss: 0.576070 	Validation Loss: 0.682396
Epoch: 11 	Training Loss: 0.569796 	Validation Loss: 0.677349
Epoch: 12 	Training Loss: 0.568266 	Validation Loss: 0.672753
Epoch: 13 	Training Loss: 0.588864 	Validation Loss: 0.731330
Epoch: 14 	Training Loss: 0.586311 	Validation Loss: 0.656463
Validation loss decreased (0.661966 --> 0.656463).  Saving model ...
Epo

In [8]:
model.load_state_dict(torch.load('model_dermatology.pt'))
model.cuda();

In [9]:
classes=["melanoma","nevus","seborrhic keratosis"]
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(3))
class_total = list(0. for i in range(3))

model.eval()
# iterate over test data
for batch_idx, (data, target) in enumerate(test_loader):
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(100):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
    print("batch index",batch_idx)

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

batch index 0
batch index 1
batch index 2
batch index 3
batch index 4
batch index 5
Test Loss: 0.660876


Test Accuracy (Overall): 72% (435/600)
